In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from torch.nn import functional as F
import random

In [ ]:
model_name = "eryk-mazus/polka-1.1b-chat"
device = 'cuda'
# device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label


def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
sentences = [
  'To jest zwykłe polskie zdanie.',
  'This is a normal English sentence.',
  'iweryuiiu hrfw3eieur fr'
]

print ()
for s in sentences:
    print (s, sentence_prob(s))


To jest zwykłe polskie zdanie. -34.943394
This is a normal English sentence. -39.47058
iweryuiiu hrfw3eieur fr -105.23458


In [ ]:
s1= """wprost|wyprosty|wyprostu|wyprost uwielbiała|wielbił|wielbiła|uwielbił|wielbiło|uwielbiał|uwielbiało|uwielbiały
słuchać|osłuchać|słychać|usłuchać o|i|e|a|ó|ę|y|ą|u
wartościach własnych|owłosionych macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz"""

In [ ]:
s2= """babcia|babci|babusi|b gotuje|zgotowała|wygotowała|przygotowuję|zgotujemy|wygotowały|
zupę|rosołu|barszczyku|żurku|flaczków z|o|e|a|ó|ę|y|ą|u
makaronem|uszkami|grzanki|warzywo"""

In [ ]:
s3= """brat|babci|babusi z|na|pod|razem|o siostrą|siostrze|braci|kuzyni|mama|tato bawią|zabawiali|wybawiają|bawić|zabawiały|
się|sobie|ich|nas zabawkami|samochodzików|pociągi w|o|e|a|ó|ę|y|ą|u
swoim|twój|moje|duży|mały pokoju|salon|pomieszczenia|szkoła"""

In [ ]:
s1.split()

['wprost|wyprosty|wyprostu|wyprost',
 'uwielbiała|wielbił|wielbiła|uwielbił|wielbiło|uwielbiał|uwielbiało|uwielbiały',
 'słuchać|osłuchać|słychać|usłuchać',
 'o|i|e|a|ó|ę|y|ą|u',
 'wartościach',
 'własnych|owłosionych',
 'macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz']

In [ ]:
sentences = list(map(lambda x: (x, 1.0), s1.split()[0].split("|")))
sentences

[('wprost', 1.0), ('wyprosty', 1.0), ('wyprostu', 1.0), ('wyprost', 1.0)]

In [ ]:
def build_correct_sentence(s, threshold=-90, top_k = 7):
  sentences = list(map(lambda x: (x, 1.0), s.split()[0].split("|")))
  for i, words in enumerate(s.split()):
    if i == 0:
      continue
    new_sentences = []
    for s, p in sentences:
      for w in words.split("|"):
        new_s = f"{s} {w}"
        new_s_prob = sentence_prob(new_s).item()
        # print(new_s, new_s_prob)
        if new_s_prob > threshold:
          new_sentences.append((new_s, new_s_prob))
    # print("!!!",sorted(new_sentences, key=lambda x: x[1], reverse=True))
    # print("???", sorted(new_sentences, key=lambda x: x[1], reverse=True)[:top_k])
    sentences = sorted(new_sentences, key=lambda x: x[1], reverse=True)[:top_k]
    print(sentences)
  return sentences[0]

In [ ]:
build_correct_sentence(s1)

[('wprost uwielbiał', -35.42815017700195), ('wyprosty uwielbiał', -37.26580810546875), ('wyprosty uwielbiały', -37.896060943603516), ('wprost uwielbiała', -38.304649353027344), ('wyprostu uwielbiał', -38.943603515625), ('wprost wielbił', -39.18402862548828), ('wyprosty uwielbiała', -39.31786346435547)]
[('wprost uwielbiała słuchać', -44.688655853271484), ('wyprosty uwielbiały słuchać', -44.85951232910156), ('wprost uwielbiał słuchać', -45.595645904541016), ('wyprosty uwielbiała słuchać', -46.275054931640625), ('wyprosty uwielbiał słuchać', -46.4627685546875), ('wyprostu uwielbiał słuchać', -47.43802261352539), ('wyprosty uwielbiały słychać', -49.835601806640625)]
[('wyprosty uwielbiały słuchać o', -48.361656188964844), ('wprost uwielbiała słuchać o', -48.57633972167969), ('wprost uwielbiała słuchać i', -48.8944206237793), ('wyprosty uwielbiały słuchać i', -49.08535385131836), ('wprost uwielbiał słuchać i', -49.5743522644043), ('wprost uwielbiał słuchać o', -49.8699836730957), ('wyprost

('wprost uwielbiała słuchać o wartościach własnych macierzy',
 -75.9924545288086)

In [ ]:
build_correct_sentence(s2)

[('b ', -16.938385009765625), ('babcia ', -19.885433197021484), ('babcia gotuje', -21.808923721313477), ('babci gotuje', -23.04676055908203), ('babci ', -23.95623779296875), ('b przygotowuję', -24.980552673339844), ('b gotuje', -26.325157165527344)]
[('babcia gotuje zupę', -30.521533966064453), ('babci gotuje zupę', -31.16571807861328), ('b przygotowuję zupę', -34.059635162353516), ('b gotuje zupę', -35.68686294555664), ('babcia gotuje rosołu', -38.561004638671875), ('b  zupę', -38.59967803955078), ('babci gotuje rosołu', -39.2174072265625)]
[('babcia gotuje zupę z', -31.883283615112305), ('babci gotuje zupę z', -32.45012664794922), ('b przygotowuję zupę z', -35.326324462890625), ('b gotuje zupę z', -36.92342758178711), ('babcia gotuje zupę a', -37.20209503173828), ('babci gotuje zupę a', -37.501251220703125), ('babcia gotuje zupę o', -37.88325500488281)]
[('babcia gotuje zupę z makaronem', -35.74850845336914), ('babci gotuje zupę z makaronem', -36.31283187866211), ('b przygotowuję zup

('babcia gotuje zupę z makaronem', -35.74850845336914)

In [ ]:
build_correct_sentence(s3)


[('brat z', -18.60269546508789), ('babci na', -19.074886322021484), ('babci z', -19.591272354125977), ('brat na', -20.029056549072266), ('brat o', -20.82484245300293), ('babci o', -21.787132263183594), ('babci pod', -22.05988311767578)]
[('brat z siostrą', -25.28647804260254), ('brat z braci', -27.27142333984375), ('babci z siostrą', -27.84465789794922), ('brat z siostrze', -29.248435974121094), ('brat z mama', -29.348684310913086), ('babci o siostrze', -29.699806213378906), ('brat o siostrze', -29.82091522216797)]
[('brat z siostrą ', -29.862384796142578), ('babci z siostrą ', -31.3160400390625), ('brat z mama ', -33.28526306152344), ('brat z siostrą bawią', -33.40492248535156), ('brat z braci ', -35.3753776550293), ('babci z siostrą bawią', -35.618072509765625), ('brat z siostrą zabawiali', -37.44470977783203)]
[('brat z siostrą bawią się', -34.31917190551758), ('babci z siostrą bawią się', -36.78113555908203), ('brat z siostrą zabawiali się', -37.952125549316406), ('brat z siostrą  

('brat z siostrą bawią się zabawkami w swoim pokoju', -47.34925079345703)